In [1]:
import geopandas as gpd
import pandas as pd
from datetime import datetime, timedelta
from shapely.geometry import Polygon, LineString, MultiPolygon
from shapely.ops import split
import copy
import time
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import copy
import pandas as pd
import sklearn


from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier



In [2]:
burned = gpd.read_file("../data/burned/sampleData.csv")
fire_names = burned.FIRE_NAME.unique()

In [3]:
fire_df = pd.read_csv('../data/fire_features/csv/BALD', index_col=0)

fire_df.head()

,SR_B1,SR_B2,SR_B3,SR_B4,SR_B5,SR_B6,SR_B7,burnSeverity,dNBR,NDVI,...,BM_forest,BM_grassland/herbaceous,BM_other,BM_shrub,BR_agriculture,BR_developed,BR_forest,BR_grassland/herbaceous,BR_other,BR_shrub
0,8747,8971,9498,9990,12917,14142,12258,1,3,0,...,1,0,0,0,0,0,1,0,0,0
1,8603,8824,9380,9693,12586,13873,12140,1,0,0,...,1,0,0,0,0,0,1,0,0,0
2,8499,8710,9281,9739,12599,12908,11446,1,4,0,...,1,0,0,0,0,0,1,0,0,0
3,8768,9042,9699,10310,13066,14288,12587,1,8,0,...,1,0,0,0,0,0,1,0,0,0
4,8465,8706,9379,9783,12953,13477,11819,1,0,0,...,1,0,0,0,0,0,1,0,0,0


In [4]:
def drop_all(df, col, ax=0, place=False):
    aux_prefix = ['TL', 'TM', 'TR', 'CL', 'XX', 'CR', 'BL', 'BM', 'BR']
    target_cols = [pref + "_" + col for pref in aux_prefix]
    
    return df.drop(target_cols + [col], axis=ax, inplace=place)

In [5]:
prefire_cols = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'dNBR', 'NDVI']

for c in prefire_cols:
    drop_all(fire_df, c, 1, True)

In [6]:
X_train = copy.deepcopy(drop_all(fire_df, 'burnSeverity', ax=1))
scaler = preprocessing.StandardScaler().fit(X_train.values)
X_scaled = scaler.transform(X_train)
X_scaled

D:\Anaconda\envs\proj\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


array([[-0.88322126,  0.        ,  0.        , ..., -0.81795958,
        -0.10563501, -0.59763024],
       [-0.86721162,  0.        ,  0.        , ..., -0.81795958,
        -0.10563501, -0.59763024],
       [-0.84586542,  0.        ,  0.        , ..., -0.81795958,
        -0.10563501, -0.59763024],
       ...,
       [ 1.67832236,  0.        ,  0.        , ..., -0.81795958,
        -0.10563501, -0.59763024],
       [ 1.68365891,  0.        ,  0.        , ..., -0.81795958,
        -0.10563501, -0.59763024],
       [ 1.68365891,  0.        ,  0.        , ..., -0.81795958,
        -0.10563501, -0.59763024]])

In [7]:
fire_df[['burnSeverity']].value_counts()

burnSeverity
0               113958
1                 8181
2                   11
dtype: int64

In [8]:
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(fire_df['burnSeverity'].values)
x_train, x_test, y_train, y_test = train_test_split(X_scaled, encoded,test_size=0.2)
pipe = LogisticRegression()
pipe.fit(x_train, y_train.ravel())
print("Logistic Regression: "+str(pipe.score(x_test, y_test)))
importance = pipe.coef_[0]
keep = []
for i in range(len(X_train.columns)):
    print(X_train.columns[i]+": "+str(importance[i]))
    if importance[i] >= 0.02:
        keep.append(X_train.columns[i])
keep

Logistic Regression: 0.9688907081457224
elevation: 2.282398830416809
pr: 0.0
sph: 0.0
th: -1.6398435672686371
tmmn: -0.290133094915178
tmmx: 0.9007597863276144
vs: -0.7583005078603673
bi: 1.50537874077215
fm100: 0.956003130551451
vpd: -1.28744738496117
impervious: 0.016125096037773522
impervious_descriptor: -0.04879535239564033
percent_tree_cover: 0.04368638452271283
TL_elevation: -0.062240131469842905
TM_elevation: 0.09668477924163033
TR_elevation: 0.3725783432621511
CL_elevation: -0.027625457379330573
XX_elevation: 0.170639241006937
CR_elevation: 0.3438777250438258
BL_elevation: 0.07437945699433829
BM_elevation: 0.17829493474546826
BR_elevation: 0.3879055862778573
TL_pr: 0.0
TM_pr: 0.0
TR_pr: 0.0
CL_pr: 0.0
XX_pr: 0.0
CR_pr: 0.0
BL_pr: 0.0
BM_pr: 0.0
BR_pr: 0.0
TL_sph: 0.0
TM_sph: 0.0
TR_sph: 0.0
CL_sph: 0.0
XX_sph: 0.0
CR_sph: 0.0
BL_sph: 0.0
BM_sph: 0.0
BR_sph: 0.0
TL_th: -0.07289712752183942
TM_th: 0.004141603368484239
TR_th: 0.035372595763066166
CL_th: -0.0714985645114819
XX_th: 

D:\Anaconda\envs\proj\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


['elevation',
 'tmmx',
 'bi',
 'fm100',
 'percent_tree_cover',
 'TM_elevation',
 'TR_elevation',
 'XX_elevation',
 'CR_elevation',
 'BL_elevation',
 'BM_elevation',
 'BR_elevation',
 'TR_th',
 'CR_th',
 'BR_th',
 'TL_tmmn',
 'TM_tmmn',
 'TR_tmmn',
 'BL_tmmn',
 'TL_tmmx',
 'TM_tmmx',
 'TR_tmmx',
 'CL_tmmx',
 'XX_tmmx',
 'CR_tmmx',
 'CR_vs',
 'BL_vs',
 'BM_vs',
 'BR_vs',
 'CL_bi',
 'XX_bi',
 'BL_bi',
 'BM_bi',
 'BR_bi',
 'BL_fm100',
 'BM_fm100',
 'BR_fm100',
 'TL_landcover',
 'TR_landcover',
 'XX_landcover',
 'BL_landcover',
 'BM_landcover',
 'TM_impervious',
 'BM_impervious',
 'TL_impervious_descriptor',
 'TM_impervious_descriptor',
 'TR_impervious_descriptor',
 'XX_impervious_descriptor',
 'BL_impervious_descriptor',
 'BM_impervious_descriptor',
 'TM_percent_tree_cover',
 'TR_percent_tree_cover',
 'BM_percent_tree_cover',
 'agriculture',
 'developed',
 'other',
 'shrub',
 'TL_agriculture',
 'TL_other',
 'TM_agriculture',
 'TM_grassland/herbaceous',
 'TM_other',
 'TR_agriculture',
 'TR_

In [9]:
%%time
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(fire_df['burnSeverity'].values)
x_train, x_test, y_train, y_test = train_test_split(X_scaled, encoded,test_size=0.2, random_state=25)
base_1 = LogisticRegression()
log_ada = AdaBoostClassifier(base_1)
log_ada.fit(x_train, y_train)
log_ada.score(x_test, y_test)

CPU times: total: 19min 40s
Wall time: 3min 31s


0.9594760540319279

In [10]:
%%time
log_ada.score(x_test, y_test)

CPU times: total: 7.23 s
Wall time: 1.36 s


0.9594760540319279

In [11]:
y_pred = log_ada.predict(x_test)
confusion_matrix(y_test, y_pred)

array([[22515,   309,     1],
       [  678,   925,     1],
       [    1,     0,     0]], dtype=int64)

In [16]:
#this model is computationally EXPENSIVE

# %%time
# lab_enc = preprocessing.LabelEncoder()
# encoded = lab_enc.fit_transform(fire_df['burnSeverity'].values)
# x_train, x_test, y_train, y_test = train_test_split(X_scaled, encoded,test_size=0.2, random_state=25)
# base_2 = LinearSVC()
# svc_ada = AdaBoostClassifier(base_2, algorithm='SAMME')
# svc_ada.fit(x_train, y_train)
# svc_ada.score(x_test, y_test)

In [17]:
# %%time
# svc_ada.score(x_test, y_test)

In [18]:
# y_pred = svc_ada.predict(x_test)
# confusion_matrix(y_test, y_pred)

In [19]:
%%time
lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(fire_df['burnSeverity'].values)
x_train, x_test, y_train, y_test = train_test_split(X_scaled, encoded,test_size=0.2, random_state=25)
base_3 = DecisionTreeClassifier()
dtr_ada = AdaBoostClassifier(base_3)
dtr_ada.fit(x_train, y_train)
dtr_ada.score(x_test, y_test)

CPU times: total: 5.05 s
Wall time: 5.05 s


0.9900532132623823

In [20]:
%%time
dtr_ada.score(x_test, y_test)

CPU times: total: 46.9 ms
Wall time: 55.1 ms


0.9900532132623823

In [21]:
y_pred = dtr_ada.predict(x_test)
confusion_matrix(y_test, y_pred)

array([[22686,   124,     0],
       [  114,  1501,     4],
       [    1,     0,     0]], dtype=int64)

In [31]:
rest_df.dropna()

,burnSeverity,elevation,pr,sph,th,tmmn,tmmx,vs,bi,fm100,...,BR_shrub,agriculture,TL_agriculture,TM_agriculture,TR_agriculture,CL_agriculture,CR_agriculture,BL_agriculture,BM_agriculture,BR_agriculture
0,1,908,0,0,181,283,296,2,40,11,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,907,0,0,181,283,296,2,40,11,...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,900,0,0,181,283,296,2,40,11,...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,888,0,0,181,283,296,2,40,11,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,883,0,0,181,283,296,2,40,11,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407677,2,151,10,0,206,279,285,3,7,19,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
407678,1,152,10,0,206,279,285,3,7,19,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
407679,1,158,10,0,206,279,285,3,7,19,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
407680,1,168,10,0,206,279,285,3,7,19,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
#One vs rest

rest = ['ATLAS', 'BUCK', 'BUTTE', 'CALDWELL']
df_arr = []
for nm in rest:
    loc = '../data/fire_features/csv/' + nm
    local_df = pd.read_csv(loc, index_col=0)
    df_arr.append(local_df)

rest_df = pd.concat(df_arr).dropna()
prefire_cols = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'dNBR', 'NDVI']
for c in prefire_cols:
    drop_all(rest_df, c, 1, True)
    
rest_X_train = copy.deepcopy(drop_all(rest_df, 'burnSeverity', ax=1))
scaler = preprocessing.StandardScaler().fit(rest_X_train.values)
rest_X_scaled = scaler.transform(rest_X_train)
rest_X_scaled

lab_enc = preprocessing.LabelEncoder()
encoded = lab_enc.fit_transform(rest_df['burnSeverity'].values)
rest_x_train, rest_x_test, rest_y_train, rest_y_test = train_test_split(rest_X_scaled, 
                                                                        encoded,test_size=0.2, 
                                                                        random_state=25)
print("preprocessing done, training first model...")

base_1 = LogisticRegression()
log_ada = AdaBoostClassifier(base_1)
log_ada.fit(rest_x_train, rest_y_train)
score_1 = log_ada.score(x_test, y_test)
print('Score with log adaboost: {0}'.format(score_1))


base_3 = DecisionTreeClassifier()
dtr_ada = AdaBoostClassifier(base_3)
dtr_ada.fit(rest_x_train, rest_y_train)
score_3 = dtr_ada.score(x_test, y_test)
print('Score with tree adaboost: {0}'.format(score_3))



D:\Anaconda\envs\proj\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


preprocessing done, training first model...
Score with log adaboost: 0.14371674171101106
Score with tree adaboost: 0.07503069995906672


In [ ]:
#test scores with other fires
best_vars = []
scores = {
    "EXT": [],
    "RFT": [],
    "KNN": []
}

for fire_n in fire_names[0:7]:
    scores = []
    times = []
    
    file_loc = '../data/fire_features/csv/' + fire_n
    fire_df = pd.read_csv(file_loc, index_col=0)
    
    for c in prefire_cols:
        drop_all(fire_df, c, 1, True)
    
    X_train = copy.deepcopy(drop_all(fire_df, 'burnSeverity', ax=1))
    scaler = preprocessing.StandardScaler().fit(X_train.values)
    X_scaled = scaler.transform(X_train)
    X_scaled
    
    lab_enc = preprocessing.LabelEncoder()
    encoded = lab_enc.fit_transform(fire_df[['burnSeverity']].values)
    x_train, x_test, y_train, y_test = train_test_split(X_scaled, encoded,test_size=0.2)
    pipe = LogisticRegression()
    pipe.fit(x_train, y_train.ravel())
    #print("Logistic Regression: "+str(pipe.score(x_test, y_test)))
    importance = pipe.coef_[0]
    keep = []
    for i in range(len(X_train.columns)):
        #print(X_train.columns[i]+": "+str(importance[i]))
        if importance[i] >= 0.02:
            keep.append(X_train.columns[i])        
    best_vars.append(keep)
    
    t1 = time.time()
    lab_enc = preprocessing.LabelEncoder()
    encoded = lab_enc.fit_transform(fire_df[['burnSeverity']].values)
    x_train, x_test, y_train, y_test = train_test_split(X_scaled, encoded,test_size=0.2)
    clf = ExtraTreesClassifier()
    clf.fit(x_train, y_train)
    sc1 = clf.score(x_test, y_test)
    t2 = time.time()
    times.append(t2-t1)
    scores.append(sc1)
    
    t1 = time.time()
    lab_enc = preprocessing.LabelEncoder()
    encoded = lab_enc.fit_transform(fire_df[['burnSeverity']].values)
    x_train, x_test, y_train, y_test = train_test_split(X_scaled, encoded,test_size=0.2)
    clf = KNeighborsClassifier()
    clf.fit(x_train, y_train)
    sc2 = clf.score(x_test, y_test)
    t2 = time.time()
    times.append(t2-t1)
    scores.append(sc2)
    
    t1 = time.time()
    lab_enc = preprocessing.LabelEncoder()
    encoded = lab_enc.fit_transform(fire_df[['burnSeverity']].values)
    x_train, x_test, y_train, y_test = train_test_split(X_scaled, encoded,test_size=0.2)
    clf = RandomForestClassifier()
    clf.fit(x_train, y_train)
    sc3 = clf.score(x_test, y_test)
    t2 = time.time()
    times.append(t2-t1)
    scores.append(sc3)
    
    finish_msg = "Done! \nResults for {0} Fire. \n
    Extra Trees: {1}, {4} secs \n
    KNeighbors: {2}, {5} secs \n
    Random Forest: {3}, {6} secs ".format(fire_n, 
                                          scores[0], scores[1], scores[2], times[0], 
                                          times[1], times[2])
    print(finish_msg)
    
    

    
    
    